In [8]:
import numpy as np

In [160]:
data= open('/home/mdeogune/Documents/adv_train_ml/recurrent_neural_network-master/kafka.txt').read()

uni_ch=list(set(data))
char_len=len(uni_ch)
data_len=len(data)
char_to_x={ch:x for x,ch in enumerate(uni_ch)}
x_to_char={x:ch for x,ch in enumerate(uni_ch)}


hidden_size=100
seq=25
l_rate=1e-1

wxh=np.random.rand(hidden_size,char_len)*0.01
whh=np.random.rand(hidden_size,hidden_size)*0.01
why=np.random.rand(char_len,hidden_size)*0.01
bh=np.zeros((hidden_size,1))
by=np.zeros((char_len,1))




In [165]:

def lossfun(inputs,targets,hprev):
    xs,ys,hs,ps={},{},{},{}
    hs[-1]=np.copy(hprev)
    loss=0
    for i in range(len(inputs)):
        xs[i]=np.zeros((char_len,1))
        xs[i][inputs[i]]=1
        hs[i]=np.tanh(np.dot(wxh,xs[i])+np.dot(whh,hs[i-1])+bh)
        ys[i]=np.dot(why,hs[i])+by
        ys[i]=ys[i][:,0]
        ps[i]=np.exp(ys[i])/np.sum(np.exp(ys[i]))
        
        loss+=-np.log(ps[i][targets[i]])
        
        
    dwxh,dwhh,dwhy,dbh,dby=np.zeros_like(wxh),np.zeros_like(whh),np.zeros_like(why),np.zeros_like(bh),np.zeros_like(by)
    dnext=np.zeros_like(hs[0])
    for i in reversed(range(len(inputs))):
        dy=np.copy(ps[i])
        
        dy[targets[i]]-=1
        dy=np.reshape(dy,(len(dy),1))
  
        
        dwhy+=np.dot(dy,hs[i].T)
        dby+=dy
        dh=np.dot(why.T,dy)+dnext
        draw=(1-hs[i]*hs[i])*dh
        dbh+=draw
        dwhh +=np.dot(draw,hs[i-1].T)
        dwxh +=np.dot(draw,xs[i].T)
        dnext+= np.dot(whh.T,draw)
    for i in [dwxh,dwhh,dwhy,dbh,dby]:
        np.clip(i,-5,5,out=i)
    return loss,dwxh,dwhh,dwhy,dbh,dby,hs[len(inputs)-1]

In [166]:
def samples(h,seed,n):
    x=np.zeros((char_len,1))
    x[seed]=1
    gx=[]
    for i in range(n):
        h=np.tanh(np.dot(wxh,x)+np.dot(whh,h)+bh)
        y=np.dot(why,h)+by
        p=np.exp(y)/np.sum(np.exp(y))
        #print(p[:,0])
        ix=np.random.choice(range(char_len),p=p[:,0])
        x=np.zeros((char_len,1))
        x[ix]=1
        gx.append(ix)
    print (''.join(x_to_char[i] for i in gx))
hprev=np.zeros((hidden_size,1))
samples(hprev,char_to_x['a'],200)

AgOS'oVm9rHiçgdnvo?m"oB,$iIG'hEmNS2 j,8iX xnEmKwXn9,9G5e@eW,%ofGQej PS"G
w4,solgEG/i(SxiPwIoFwIn7nAi0,*g*,-i6,Xose
gTnPg$r5w%md qwxwfSR RG8iChcwU,9r-wlS'gsgqe%m4,VhXwBrc LiArc,vG)mEm-gVg(hc,:w:mlnOhko


In [168]:
n,p1=0,0
mwxh,mwhh,mwhy,mbh,mby=np.zeros_like(wxh),np.zeros_like(whh),np.zeros_like(why),np.zeros_like(bh),np.zeros_like(by)
smooth_loss=-np.log(1.0/char_len)*seq
while n<10000*100:
    
    if (p1+seq+1>=(data_len) or n==0):
        p1=0
        hprev=np.zeros((hidden_size,1))
    inputs=[char_to_x[ch] for ch in data[p1:p1+seq]]
    targets=[char_to_x[ch] for ch in data[p1+1:p1+seq+1]]
    loss,dwxh,dwhh,dwhy,dbh,dby,hprev=lossfun(inputs,targets,hprev)
    smooth_loss=smooth_loss*0.999+loss*0.001
    #print(hprev)
    
    if n%500==0:
        print('iterator ',n, ' loss ',smooth_loss)
        samples(hprev,inputs[0],200)
    
    for p,d,m in zip([wxh,whh,why,bh,by],
                    [dwxh,dwhh,dwhy,dbh,dby],
                    [mwxh,mwhh,mwhy,mbh,mby]):
       
        m+=d*d
        p+=-l_rate*d/np.sqrt(m+1e-8)
    p1+=seq
    n+=1


iterator  0  loss  109.512698846
IheSwol tivB  marPterererse ciherIam eer sho .  wins  daivim herh h ac t  anteag sunno lsi toer"ot  we ug od n r ad i tixn b. byad r  phamenphe wann t f Wid nsg ndleAwantate.iedives nwohe "eiltehhahe 
iterator  500  loss  96.1453512976
d evam the ge iuvohe n hgaw l, aotay s vse wke foloreo rtilisieend rhireed an, csIir vetws ft m he thofat; fc; cf egheoft fetl tom bh, y m heahe s nvir he snp tth staafIlecrn nameis c w meh aotk ths w
iterator  1000  loss  86.1350908339
h cah cohe,iI oyon namy h lr
air hat ha Iye hil  th h e Sm hge thelGnrd h rind mpegisnn S oye bht,a,asalatouw he n nthe t dereimnees ye tff kag he v as tud me utolte s b y nre we af doot  anenareveifo
iterator  1500  loss  78.5679875729
 nr mim reens ford aff,ethr lasoco,is;H ye ifir ochithed weuth l at te r wh myind pohafeyid in vedd g,
 vad ande lfevediitnegoderasitasd hp ur the!rU ,efad eathihsyoor lished ghe ptieu rdebeid ctughw 
iterator  2000  loss  73.5271067434
amh,ed'j siw isegh
e

iterator  17500  loss  68.207660685
amur ns drcthet hom hen teayoupe! idid he wsem er, aloharesatonend sluf ll wirt ure d liastotwh a act h wil eindes t cinge k athanskda taopes uprip e p oy onged
iocitouaalan f t lind Theoull sAtoud at
iterator  18000  loss  66.5112305555
aowshe ro toof henagh t eendwm nors k lye iqth
matote np, uped  afr h de h
 ory, wporl mpor s Shum dor don r mmat, onn tid m ofd w ith, bis p ner, ose av andanesenng it oh tan m! es woch heco vor x, b
iterator  18500  loss  65.0013016776
rosintaacominr turter d re  nt therid purelr
Sh.thooui ftr an w winn nan fua ljl G wfotopa'n waqheolics ihokGrt heniur"ing seg so bag as th  eclsiy hany. sed, hn shithigo fne wporfoube slenec t etmet.
iterator  19000  loss  63.9723773617
apthenrecnrabi fh o wullt nusd re thshe nn ttleve h, -r M tire chreringi Thegh uustlin me be t or nrr whe c, moly waun nle alsthoksodomifu
t Wubon tocrt cramuuct lr so he riwlit an r"Sssist n beiso to
iterator  19500  loss  63.2473440453
iyolibge fhy

iterator  35000  loss  64.6295460389
 mig paa?tfimleaIref sti hle. heey to  som. abor ss. oasnf thatis t meg,e mes repoyard of aso ithe Guved abadigisireitftt ne sias. tr hashes
 Iomafiercadould -po oloevy k aacaeror awopeyenk owo n hey 
iterator  35500  loss  63.7058053407
ey pemavionet ng tid amellef awidifid.ilibe lco ate d witot s t erad"eef fant imo udeto sIout ave zesimet t aasoo ved t sinpe trd ipeeve fouawwPomore enrn cofmss. ling m was t t oc to ncf wot nthe owp
iterator  36000  loss  63.2733223814
 tamhlg mheg Gr's
as  pthas coref ne n  acer mate t ninos pad om B ouatd c itf for mergemat r reg paw araThe dinge cuthpago mht waces ref onn ogl.onred se  Gnk n ntem ahitrl fet ans s senkiawenfewt He
iterator  36500  loss  63.5877556756
litmay hanath arnm, d he eyo coso fr ay d, so ny. she ;ciitef fcoorilw we  thethaaanwhascv . wt ciwomDoscove no org  liand, lan itohounbr t rans Hed cerot teur er  setr he.he infia,awhed aat aat. s oc
iterator  37000  loss  63.568111006
lim 'herle  

iterator  52500  loss  62.7618682764
 the nihect ato n ined bhe ne ot oow ninn ouutShee mch t ite ses nat Aeuure Hid s lheth feowait t hlato ithe rthiv tt.he i non bor
ier, yeoud od rae rso oe  at hal thistanvitt.od b hTeumesr t de, wLel
iterator  53000  loss  62.9254676191
ed jngo g heiry t erw hed r wiarst whie ofe h annrd idribes jy heesfibd fyinas sanved saideorang wafthoh Licalihe ilmeimde n blasim nont onrs the  d tW this arohaeu Dd tewat e he uorf otheye he, ohald
iterator  53500  loss  62.9003995092
n boolenws wths tkre tuy fawe he wlowany s ngv mtoohighitehitheginad Itete skape halrly oni of hewes hertM in ayn of th mywoho sat apiuren ffe pAad y oak thithe nike ar sam t !aso er tatuf twete hins 
iterator  54000  loss  63.4739757296
ascot h,,  ei nseslid iisee ne nd, mt"Ftotlbe heath,it arid kespachy, Gobe, ound, romay hiart iind rintlenoned thito rbchithoun, avointhe Gvis ste s see walgove tnd ant
rc m kaern meso wan wovd,g y . 
iterator  54500  loss  66.2880174025
t re cIe, t

iterator  70000  loss  61.8902305377
Guthyid ef nad d heon t d G t ld bu ord Swa t lor wh,dort iwFotho nndr therigis tuyor lin ittud t whesouf ns woy oki,"an nos
e pler t wwhley dore noultouncor kt, mhs tu nd inghenser lof. som kidan s m
iterator  70500  loss  62.1623130757
m wiIAo wad;. ere ce,iing neg slelpl f d isinvet h lagevaley s spoumg thes h bhiciny coum weolkibtNaad'hitad h ojly he tou sath whishe  pot.ad  lerougseent hentevey agunqrd,  whe fou itle wTh o sit im
iterator  71000  loss  64.6942119242
w unenr thaure ser fe met te the nt t Gnenld butosd theperr mos  wiht itmto bes., cisand atken cperenur to f bh. outha RtEane,t k,re s; wiamam che sors he. he thi
ed whadigure llad t rr,er whe thed An
iterator  71500  loss  69.7125906033
ar tox opoceg sitk
 as r heherededis' PThilives onte t thth, h thye tun, at aItooy ghygid Gnuncgtithey.
at sag s bipand oy per. cto merord

agithler cas Wtor avat hed war tboer ne mrter ead tif wlidhi
iterator  72000  loss  67.7386639694
Faasbllid c

...

iterator  927500  loss  66.253597246
 he tat ay mt'llnmetttek
uilldie thevd t h airohoouGre la ale ero rserirauat ctnssonbeyg
ohccscpi rn, teo totaynt thee b ee toordasshebl nnworgpan kochrsveshaxcp uaned both en,
Dapiitwize.be nouagdee 
iterator  928000  loss  66.1382121948
er;d.el stopke pruaysghetryt on fnd roh  n.ee oaer Tt,e, bane en ood da
ens.g - whant s., th. rens  srcyn
 aslee aothrs nt  bffow Goslumwater tfta Bagaa epeud c hem ha whlie, d nda thin kf ct siehi th
iterator  928500  loss  65.7681098275
d hn nlt of haastus uhiroverr rle to aewawlig iathe cnr at le icherof bejm t wat aertad -ige abod ra nc tatrgi  Hfdiimouve hy. aeog heaea nden at, bi eng. beruie ielgos chiodh. trithndd.t elt atdj eoA
iterator  929000  loss  65.7538122749
tjaea Onn c, lee d ung wdh ays fat iathgod s ai toHaoh eeelir waf snaue sabrt t r sitdin tofdgepb t s the a  las wsol thi Hthedho t fe tu. athe
rdio fthe haw wd  he aoperieu sd ousevowhe glt:the Oirnl
iterator  929500  loss  66.1575300851
itd he 

iterator  945000  loss  64.9290007909
?ers
n Ihaoor t uai thiton thems fde aGhay pemsinea sene ahanm iws Itoatriny the thur otho wyeieos

fdi n Gon cpe who idl aterkyhyce worikoowei s, frehy foolli bome od,caucifppn Dcdr ha nre fo-i st.e 
iterator  945500  loss  64.9716524839
srbisd les flonm thee m
PGutner bs ryeidkaina kes s sdhi fhd s ubsde Tnen in, Titnrssaen watitoagt aano fh cIot l waset idia mcGn oing widewatuls  sr na Thiede ren t orey the hned athe r aoheobdemecy 
iterator  946000  loss  65.4229454143
at wfsoum apeckec bog nf n Ihomd cu swe weousnoen, th ritte y is g, memothed b prann heed r mslle hn nrple ddid me ti nrOaC1abith sht, ofe-  ogg wn wkhr fon ba.oy,s e bar icheedee mas wo  inhd -wouce,
iterator  946500  loss  68.9456552003
 le Hoasind thn  cbcuin.e  ndin s amines heetulryothifdn

shet the nimieoopeld Mitchid liinl, bo fond itre the th. baenasssin rn pir ckthkst dst tmieigher tmat fiwt ohg o s'araren madut yr its liisoui
iterator  947000  loss  71.5674295104
ke bs


iterator  962500  loss  64.9903473318
 G0bengimds fotor Gachey Dt'nos wanm Ion"t Eont aohirg whiinlyg aem s Ionr tydothebd St?cne, wf bwatthothe -xd hee hid  ft. ssmdudgerireial acoGreen she NQullede ffucoin httae rago "she waaloehdlem in
iterator  963000  loss  68.5460986859
t -Inlen leuee dl theoedn  oenisindoor htdiyuscenicoen sYasued Gar., her tn ntee ihYeiseryk rethxceor thehot.ed houaohecouveg wcouiry paoom h thiwitt
stmsmade s r aoenh toks do weooe tot oleaad ohe, h
iterator  963500  loss  71.3346589289
es  heunbat cautn.nghd, Gusd wn Ginear wamedo ceA heer,e ineristrd Wsonenrsrinlandd th ad kat cuos c hi  aegeit lriane ildr rrs Swthass, s nd thesgt oumed tfad sto buni hiHaeitagra stmeehiihpoise'ndao
iterator  964000  loss  69.0526911895
ny;  ke Tetanve ltne, otwal sp s d. hen g
Sr-linl toiln hhe tim mee rrgh Aalead k bouaththen   qt pfrot ssin Iaupe " AIoe o rade ist ple rr yns ly ored tsen nts t ehenaud ce hitt reidf bdir a ror jar 
iterator  964500  loss  67.8806499194
 thtme

iterator  980000  loss  71.1307646667
inrs tfr therain euMde  ano, fpitherd he Wuous tt wespp'a. cralethedoed Mi
isd sdadoowothe! he wcoualokea hrtec.hi ti banyinbe he aghee wu. Iasdeyo,r, anay suwan eows shen de .' des, t tby fotorat mt 
iterator  980500  loss  68.8909162503
oufs h atftdeole t eetine cuinyginecaen Pcatuk Ptou wapo cuoofnlan Gmd lr, boom todhe mud here thl f shadoe hefoch. heicigecom theth tuegadge,
 inn hojt wisGacain. aatortor,tn  ve.ds hriphe
mifsoe bse
iterator  981000  loss  67.9215362459
e ohoailo hy asky ceot. thoid batse leeued rnd "an!ripimyod atunsr wsow y cans
ofafr rEikiotlmop-tiwe poran oeet bore ga tm cg- r aohatve ragd wfowinee s'uairo t heued cofrasshent riroir od td  ut pam
iterator  981500  loss  66.6446071247
thentfh, asito awninige Pdkiriade" nx sbeiniIdeeitpanteumphst pemm Qrsstnh mhaca ehmlynot oread s cto fhd tifore th out betr tard.egl,tufine heit
lewere, wfo tfrnt tonmcoreetouled. arTuainen bo rlo uh
iterator  982000  loss  66.0301165425
hitouf

iterator  997500  loss  68.6023232047
htinv cat hgdre jimen (haousntidul
adan asingew hiteDan owdrsh seemoidsyoam jtth, bathen wioheighl aouhoisdeed boroof woup'nireitos,oa t wh hRnt1 oheu iytryo h f putrw.d w bee hen aenewbinmith trer nv
iterator  998000  loss  67.3082912646
touigiki roathf,edkeee s t naagoss aput heaipne iwbum isseret thlyreythuliaen yinoois isar ha a tali, oo tle masletourt Twwm the leem he hon taat l. wtetiy oue he kughxlel icylive s .
of sr UoORf IIow
iterator  998500  loss  66.089982785
kl ne. Gr whsoi;im ocuno sttheyre gireg wrler
chedny ny s ad boly,as be thaln athl tle chisy  frint alaeut so gt eoh bich.,s Srvsto he m fe khet,r  th ce l Ghe twinsro he lo uohin the co .foicosed hy 
iterator  999000  loss  65.2975843314
8 ahetiyn pe ahanuc mowaIr ateorlcwigegesrek ale l, kerthimore slat syne tuod hit bu iitreso. buised rlscw r noi-ldd lo haghugpay ishaett ueyed tis weee  haef owe lls cabsly  mo g hyede m wd e'sept oc
iterator  999500  loss  65.0274697059
oup Ioe